# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links 
* [Planet Client V2 Documentation](https://github.com/planetlabs/planet-client-python)
* [Planet Data API reference](https://developers.planet.com/docs/apis/data/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API.

The basic workflow for interacting with the Data API is:
1. search item types based on filters
1. activate assets
1. download assets

## Set up

In order to interact with the Planet API using the client, we need to import the necessary packages & define helper functions.

In [1]:
from datetime import datetime
from planet import Auth
from planet import Session, data_filter
import json
import time
import os

# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

We next need to create a `client` object registered with our API key. The API key will be automatically read from the `PL_API_KEY` environment variable if it exists. If not, you can provide it below. 

In [2]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = 'PASTE_API_KEY_HERE'
    os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

Let's also read in a GeoJSON geometry into a variable so we can use it during testing.

In [3]:
with open("data/san-francisco.json") as f:
    geom = json.loads(f.read())

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

### Filters

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API.


In [4]:
# Define the filters we'll use to find our data

item_types = ["REOrthoTile", "PSScene"]

geom_filter = data_filter.geometry_filter(geom)
clear_percent_filter = data_filter.range_filter('clear_percent', 90)
date_range_filter = data_filter.date_range_filter("acquired", gt = datetime(month=1, day=1, year=2017))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.1)

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter, cloud_cover_filter])

In [5]:
combined_filter

{'type': 'AndFilter',
 'config': [{'type': 'GeometryFilter',
   'field_name': 'geometry',
   'config': {'type': 'Polygon',
    'coordinates': [[[-122.47455596923828, 37.810326435534755],
      [-122.49172210693358, 37.795406713958236],
      [-122.52056121826172, 37.784282779035216],
      [-122.51953124999999, 37.6971326434885],
      [-122.38941192626953, 37.69441603823106],
      [-122.38872528076173, 37.705010235842614],
      [-122.36228942871092, 37.70935613533687],
      [-122.34992980957031, 37.727280276860036],
      [-122.37773895263672, 37.76230130281876],
      [-122.38494873046875, 37.794592824285104],
      [-122.40554809570311, 37.813310018173155],
      [-122.46150970458983, 37.805715207044685],
      [-122.47455596923828, 37.810326435534755]]]}},
  {'type': 'RangeFilter', 'field_name': 'clear_percent', 'config': {'gt': 90}},
  {'type': 'DateRangeFilter',
   'field_name': 'acquired',
   'config': {'gt': '2017-01-01T00:00:00Z'}},
  {'type': 'RangeFilter', 'field_name': '

Now let's build the request:

In [6]:
async with Session() as sess:
    cl = sess.client('data')
    request = await cl.create_search(name='planet_client_demo',search_filter=combined_filter, item_types=item_types)

In [7]:
request

{'__daily_email_enabled': False,
 '_links': {'_self': 'https://api.planet.com/data/v1/searches/0c8832ef98504fb8a37463f653aa8f9a',
  'results': 'https://api.planet.com/data/v1/searches/0c8832ef98504fb8a37463f653aa8f9a/results'},
 'created': '2023-03-31T14:46:44.252598Z',
 'filter': {'config': [{'config': {'coordinates': [[[-122.47455596923828,
        37.810326435534755],
       [-122.49172210693358, 37.795406713958236],
       [-122.52056121826172, 37.784282779035216],
       [-122.51953124999999, 37.6971326434885],
       [-122.38941192626953, 37.69441603823106],
       [-122.38872528076173, 37.705010235842614],
       [-122.36228942871092, 37.70935613533687],
       [-122.34992980957031, 37.727280276860036],
       [-122.37773895263672, 37.76230130281876],
       [-122.38494873046875, 37.794592824285104],
       [-122.40554809570311, 37.813310018173155],
       [-122.46150970458983, 37.805715207044685],
       [-122.47455596923828, 37.810326435534755]]],
     'type': 'Polygon'},
    

In [9]:
# Search the Data API

# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100. Here, we're setting our result limit to 50.
async with Session() as sess:
    cl = sess.client('data')
    items = cl.run_search(search_id=request['id'], limit=50)
    item_list = [i async for i in items]

Now, we can iterate through our search results.

In [10]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

20230327_182710_28_247f PSScene
20230327_182707_89_247f PSScene
20230327_184115_35_2479 PSScene
20230327_184119_73_2479 PSScene
20230327_184117_54_2479 PSScene
20230327_180039_26_2447 PSScene
20230327_180037_08_2447 PSScene
20230326_180347_13_24d0 PSScene
20230326_180344_75_24d0 PSScene
20230326_180602_94_24c7 PSScene
20230326_180600_59_24c7 PSScene
20230325_183720_99_247a PSScene
20230325_183723_19_247a PSScene
20230325_175550_47_2460 PSScene
20230325_175548_26_2460 PSScene
20230324_182240_80_2251 PSScene
20230324_182242_84_2251 PSScene
20230317_183408_23_2486 PSScene
20230315_175637_54_2447 PSScene
20230315_175639_71_2447 PSScene
20230315_183426_77_249a PSScene
20220511_183556_26_249c PSScene
20220524_180049_18_2442 PSScene
20220611_183304_29_2474 PSScene
20220622_183136_03_2470 PSScene
20220622_183138_31_2470 PSScene
20220625_183343_26_2474 PSScene
20220624_183354_67_2484 PSScene
20220624_183352_39_2484 PSScene
20220713_182927_06_2470 PSScene
20230120_180119_31_24cf PSScene
20230121

If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

In [11]:
with open('output/results.json','w') as f:
    jsonStr = json.dumps(item_list)
    f.write(jsonStr)
    f.close()

This GeoJSON file can be opened and viewed in any compatible application.

## Assets and downloads

After a search returns results, the Python client can be used to check for assets and initiate downloads. Let's start by looking at one item and the assets available to download for that item.

For more information on Items and Assets, check out [Items & Assets](https://developers.planet.com/docs/apis/data/items-assets/) on the Planet Developer Center.

In [12]:
# As an example, let's look at the first result in our item_list and grab the item_id and item_type:
item = item_list[0]

print(item['id'], item['properties']['item_type'])

20230327_182710_28_247f PSScene


In [13]:
# The list of assets for an item that a user has access to can be retrieved with `permissions`
item['_permissions']

['assets.basic_analytic_4b:download',
 'assets.basic_analytic_4b_rpc:download',
 'assets.basic_analytic_4b_xml:download',
 'assets.basic_analytic_8b:download',
 'assets.basic_analytic_8b_xml:download',
 'assets.basic_udm2:download',
 'assets.ortho_analytic_4b:download',
 'assets.ortho_analytic_4b_sr:download',
 'assets.ortho_analytic_4b_xml:download',
 'assets.ortho_analytic_8b:download',
 'assets.ortho_analytic_8b_sr:download',
 'assets.ortho_analytic_8b_xml:download',
 'assets.ortho_udm2:download',
 'assets.ortho_visual:download',
 'assets.ps3b_analytic:download',
 'assets.ps3b_analytic_dn:download',
 'assets.ps3b_analytic_dn_xml:download',
 'assets.ps3b_analytic_xml:download',
 'assets.ps3b_basic_analytic:download',
 'assets.ps3b_basic_analytic_dn:download',
 'assets.ps3b_basic_analytic_dn_rpc:download',
 'assets.ps3b_basic_analytic_dn_xml:download',
 'assets.ps3b_basic_analytic_rpc:download',
 'assets.ps3b_basic_analytic_xml:download',
 'assets.ps3b_basic_udm:download',
 'assets.ps

In [14]:
# Define our item_id, and item_type

item_id = item['id']
item_type = item['properties']['item_type']

There are a few steps involved in order to download an asset using the Planet Python Client:

* **Get Asset:** Get a description of our asset based on the specifications we're looking for
* **Activate Asset:** Activate the asset with the given description
* **Wait Asset:** Wait for the asset to be activated
* **Download Asset:** Now our asset is ready for download!

Let's go through these steps below. We'll do this for our analytic asset, as well as the analytic_xml asset.

In [16]:
# Analytic Asset
async with Session() as sess:
    cl = sess.client('data')
    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='ortho_analytic_4b')
    # Activate Asset
    await cl.activate_asset(asset=asset_desc)
    # Wait Asset
    await cl.wait_asset(asset=asset_desc)
    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory='output', overwrite=True)

output/20230327_182710_28_247f_3B_AnalyticMS.tif: 100%|█████| 601k/601k [00:45<00:00, 13.8MB/s]


In [17]:
# Analytic XML Asset
async with Session() as sess:
    cl = sess.client('data')
    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='ortho_analytic_4b_xml')
    # Activate Asset
    await cl.activate_asset(asset=asset_desc)
    # Wait Asset (this may take some time!)
    await cl.wait_asset(asset=asset_desc)
    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory='output', overwrite=True)

output/20230327_182710_28_247f_3B_AnalyticMS_metadata.xml: 100%|█| 0.01k/0.01k [00:00<00:00, 2.


Congratulations! Both the `ortho_analytic_4b` and `ortho_analytic_4b_xml` assets should be saved in our `output` directory.

## Saved Searches

The Data API client can also help in managing saved searches on the Planet Platform.

In [19]:
async with Session() as sess:
    cl = sess.client('data')
    searches = cl.list_searches()
    searches_list = [i async for i in searches]

In [20]:
# We may have a lot of saved searches!

len(searches_list)

100

View your saved searches. Here we're viewing our first 50 results.

In [21]:
for search in searches_list[:50]:
    print(search['id'], search['name'])

0c8832ef98504fb8a37463f653aa8f9a planet_client_demo
e1bf9d24d195427284a7899d94e9d4d9 e1bf9d24d195427284a7899d94e9d4d9
f5fddc1399f34aea8a72cfbe6d200ad9 f5fddc1399f34aea8a72cfbe6d200ad9
bcf953420177451d82caacd6de7d7009 bcf953420177451d82caacd6de7d7009
f39f83c804694a07a9f5f43b32893250 f39f83c804694a07a9f5f43b32893250
11a602c144ab49a48b21d2c9ce82046c 11a602c144ab49a48b21d2c9ce82046c
8c0688e66ff14ee0b475e5babe15b808 8c0688e66ff14ee0b475e5babe15b808
dd73eb107fcd457cb6a8cd10e9338030 dd73eb107fcd457cb6a8cd10e9338030
9507024fbbb94af09567e2059d4cd3c8 9507024fbbb94af09567e2059d4cd3c8
630395709d974cbe85eb9feb1ee93f7d 630395709d974cbe85eb9feb1ee93f7d
f43fa8595b8d4f5c866567887f72973c f43fa8595b8d4f5c866567887f72973c
e8bcf17286c54e69b98e70f6143ecda9 e8bcf17286c54e69b98e70f6143ecda9
7f8440f5cf8b4ed8b1757dc2295e963c 7f8440f5cf8b4ed8b1757dc2295e963c
43c46cc0fe074483a0e07a625efd8101 43c46cc0fe074483a0e07a625efd8101
144d242ed8734d8fa1e8f24230ad815c 144d242ed8734d8fa1e8f24230ad815c
78bce0ad640f48fdb02fd983

We can find the saved search we're looking for by filtering on our search id:

In [22]:
async with Session() as sess:
    cl = sess.client('data')
    search = await cl.get_search(searches_list[0]['id'])

In [23]:
search

{'__daily_email_enabled': False,
 '_links': {'_self': 'https://api.planet.com/data/v1/searches/0c8832ef98504fb8a37463f653aa8f9a',
  'results': 'https://api.planet.com/data/v1/searches/0c8832ef98504fb8a37463f653aa8f9a/results'},
 'created': '2023-03-31T14:46:44.252598Z',
 'filter': {'config': [{'config': {'coordinates': [[[-122.47455596923828,
        37.810326435534755],
       [-122.49172210693358, 37.795406713958236],
       [-122.52056121826172, 37.784282779035216],
       [-122.51953124999999, 37.6971326434885],
       [-122.38941192626953, 37.69441603823106],
       [-122.38872528076173, 37.705010235842614],
       [-122.36228942871092, 37.70935613533687],
       [-122.34992980957031, 37.727280276860036],
       [-122.37773895263672, 37.76230130281876],
       [-122.38494873046875, 37.794592824285104],
       [-122.40554809570311, 37.813310018173155],
       [-122.46150970458983, 37.805715207044685],
       [-122.47455596923828, 37.810326435534755]]],
     'type': 'Polygon'},
    

# Statistics

The Data API client can also help report statistical summaries of the amount of data in the Planet API.

In [24]:
# Here, we can look at the stats for the search we've been working with:

async with Session() as sess:
    cl = sess.client('data')
    stats = await cl.get_stats(interval='year', search_filter=combined_filter, item_types=item_types)

In [25]:
indent(stats)

{
  "buckets": [
    {
      "count": 21,
      "start_time": "2017-01-01T00:00:00.000000Z"
    },
    {
      "count": 388,
      "start_time": "2018-01-01T00:00:00.000000Z"
    },
    {
      "count": 615,
      "start_time": "2019-01-01T00:00:00.000000Z"
    },
    {
      "count": 731,
      "start_time": "2020-01-01T00:00:00.000000Z"
    },
    {
      "count": 656,
      "start_time": "2021-01-01T00:00:00.000000Z"
    },
    {
      "count": 447,
      "start_time": "2022-01-01T00:00:00.000000Z"
    },
    {
      "count": 64,
      "start_time": "2023-01-01T00:00:00.000000Z"
    }
  ],
  "interval": "year",
  "utc_offset": "+0h"
}
